<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>1. Problem Statement</h2>
</div>

### Forecasting the visitor for restaurants


<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>2. Load Data</h2>
</div>

In [55]:
#load the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
# Set Options
pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 500)
pd.set_option('expand_frame_repr', False)
%matplotlib inline
warnings.filterwarnings("ignore")

In [13]:
%%time
#load the datasets
#each resaturant has unique AIR id and HPG ig , we have store rleated data from two systems AIR, HPG
air_store_info=pd.read_csv("Data_Restaurant_Visitor_Forecasting/air_store_info.csv")   #air store info
hpg_store_info=pd.read_csv("Data_Restaurant_Visitor_Forecasting/hpg_store_info.csv")   #hpg stor einfo
sample_submission=pd.read_csv("Data_Restaurant_Visitor_Forecasting/sample_submission.csv")
date_info=pd.read_csv("Data_Restaurant_Visitor_Forecasting/date_info.csv")
hpg_reserve=pd.read_csv("Data_Restaurant_Visitor_Forecasting/hpg_reserve.csv")
air_reserve=pd.read_csv("Data_Restaurant_Visitor_Forecasting/air_reserve.csv")
air_visit_data=pd.read_csv("Data_Restaurant_Visitor_Forecasting/air_visit_data.csv")
store_id_relation=pd.read_csv("Data_Restaurant_Visitor_Forecasting/store_id_relation.csv")

CPU times: total: 1.2 s
Wall time: 1.75 s


In [14]:
air_store_info.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [15]:
hpg_store_info.head()

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


In [16]:
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [17]:
hpg_reserve.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [18]:
air_visit_data.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [19]:
store_id_relation.head()

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7


### Inference 
* there are total 6 dataset with hotel visitors data, hotel location, date, and all

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>3.Understanding Data - Univariate Analysis</h2>
</div>

### 
Before attempting to solve the problem, it's very important to have a good understanding of data.

The goal of this section is to:
- Get shape and summary of data frames
- Get various statistics of data
- Get inferences or action items for next step

In [68]:
def understanddata(df):
    print(f"dimesnion of dataset \n{df.shape}\n\n") #dimension of dataset
    print(f"summary of dataset \n {df.describe()}\n\n") #summary of dataset
    print(f"{df.info()}\n\n")
    stats=[]
    for col in df.columns:
        stats.append((col,df[col].nunique(),df[col].isnull().sum()*100/df.shape[0],df[col].value_counts(normalize=True,dropna=False).values[0]*100,df[col].dtype))

    stats_df=pd.DataFrame(stats,columns=['feature','unique','perc of missing values','perc of values of the  biggest category','col dtype'])
    print(f"Statics of dataset are- \n\n{stats_df.sort_values('perc of missing values',ascending=False)}\n\n")    
                          
                     

#### Store info data -Air, HPG 

In [69]:
#store info data
understanddata(air_store_info)


dimesnion of dataset 
(829, 5)


summary of dataset 
          latitude   longitude
count  829.000000  829.000000
mean    35.647042  137.415311
std      2.084659    3.650971
min     33.211967  130.195555
25%     34.695124  135.341564
50%     35.658068  139.685474
75%     35.694003  139.751599
max     44.020632  144.273398


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   air_store_id    829 non-null    object 
 1   air_genre_name  829 non-null    object 
 2   air_area_name   829 non-null    object 
 3   latitude        829 non-null    float64
 4   longitude       829 non-null    float64
dtypes: float64(2), object(3)
memory usage: 32.5+ KB
None


Statics of dataset are- 

          feature  unique  perc of missing values  perc of values of the  biggest category col dtype
0    air_store_id     829                     0.0                   

In [70]:
understanddata(hpg_store_info)

dimesnion of dataset 
(4690, 5)


summary of dataset 
           latitude    longitude
count  4690.000000  4690.000000
mean     35.810261   137.675816
std       2.138755     3.197233
min      33.311645   130.339313
25%      34.692109   135.498859
50%      35.659214   139.495733
75%      35.703381   139.737998
max      43.774846   143.714585


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4690 entries, 0 to 4689
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   hpg_store_id    4690 non-null   object 
 1   hpg_genre_name  4690 non-null   object 
 2   hpg_area_name   4690 non-null   object 
 3   latitude        4690 non-null   float64
 4   longitude       4690 non-null   float64
dtypes: float64(2), object(3)
memory usage: 183.3+ KB
None


Statics of dataset are- 

          feature  unique  perc of missing values  perc of values of the  biggest category col dtype
0    hpg_store_id    4690                     

<div class="alert alert-info" style="padding:0px 10px; border-radius:5px;"><h3 style='margin:10px 5px'> Inferences:</h3>
</div>

* Air has 859 store info and HPG has 4690
* Both dataset has 5 unique columns
* No missing data in both datasets

#### Store Visit Data- Air

In [82]:
understanddata(air_visit_data)

dimesnion of dataset 
(252108, 3)


summary of dataset 
             visitors
count  252108.000000
mean       20.973761
std        16.757007
min         1.000000
25%         9.000000
50%        17.000000
75%        29.000000
max       877.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252108 entries, 0 to 252107
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   air_store_id  252108 non-null  object
 1   visit_date    252108 non-null  object
 2   visitors      252108 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 5.8+ MB
None


Statics of dataset are- 

        feature  unique  perc of missing values  perc of values of the  biggest category col dtype
0  air_store_id     829                     0.0                                 0.189205    object
1    visit_date     478                     0.0                                 0.316928    object
2      visitors     204                     0.0

In [83]:
#min amd max date of vsist
min,max=air_visit_data.visit_date.min(),air_visit_data.visit_date.max()
print(min,max)
print(f"days {(pd.to_datetime(min)-pd.to_datetime(max)).days}")

2016-01-01 2017-04-22
days -477


In [84]:
# Stores present in air visit data not present in air store info
np.setdiff1d(air_visit_data.air_store_id,air_store_info.air_store_id)

array([], dtype=object)

<div class="alert alert-info" style="padding:0px 10px; border-radius:5px;"><h3 style='margin:10px 5px'> Inferences:</h3>
</div>

* Air has total 252108 visitor and 478 unique dates
* all store id's(829) from visitors are present in AIR store dataset
* This is the main training data
* no missing data, total days data 477


### Reservation Data - HPG , AIR

In [88]:
understanddata(hpg_reserve)

dimesnion of dataset 
(2000320, 4)


summary of dataset 
        reserve_visitors
count      2.000320e+06
mean       5.073785e+00
std        5.416172e+00
min        1.000000e+00
25%        2.000000e+00
50%        3.000000e+00
75%        6.000000e+00
max        1.000000e+02


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000320 entries, 0 to 2000319
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   hpg_store_id      object
 1   visit_datetime    object
 2   reserve_datetime  object
 3   reserve_visitors  int64 
dtypes: int64(1), object(3)
memory usage: 61.0+ MB
None


Statics of dataset are- 

            feature  unique  perc of missing values  perc of values of the  biggest category col dtype
0      hpg_store_id   13325                     0.0                                 0.057741    object
1    visit_datetime    9847                     0.0                                 0.526316    object
2  reserve_datetime   11450            

In [97]:
#min and max date time reserve date time 
min,max=hpg_reserve.reserve_datetime.min(),hpg_reserve.reserve_datetime.max()
print(min,max)
print(f"days {(pd.to_datetime(min)-pd.to_datetime(max)).days}")

2016-01-01 00:00:00 2017-04-22 23:00:00
days -478


In [98]:
#min and max date time  for hpg visit 
min,max=hpg_reserve.visit_datetime.min(),hpg_reserve.visit_datetime.max()
print(min,max)
print(f"days {(pd.to_datetime(min)-pd.to_datetime(max)).days}")

2016-01-01 11:00:00 2017-05-31 23:00:00
days -517


In [90]:
#stores present in hpg stores and reerve data
np.setdiff1d(hpg_reserve.hpg_store_id,hpg_store_info.hpg_store_id)

array(['hpg_001112ef76b9802c', 'hpg_0023b061a9f3279a',
       'hpg_0025a7a2ddb76a01', ..., 'hpg_fffa3b36bc525136',
       'hpg_fffa98e9f58176a9', 'hpg_fffc097dce87af3e'], dtype=object)

In [92]:
# AIR reservation data
understanddata(air_reserve)

dimesnion of dataset 
(92378, 4)


summary of dataset 
        reserve_visitors
count      92378.000000
mean           4.481749
std            4.919669
min            1.000000
25%            2.000000
50%            3.000000
75%            5.000000
max          100.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92378 entries, 0 to 92377
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   air_store_id      92378 non-null  object
 1   visit_datetime    92378 non-null  object
 2   reserve_datetime  92378 non-null  object
 3   reserve_visitors  92378 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 2.8+ MB
None


Statics of dataset are- 

            feature  unique  perc of missing values  perc of values of the  biggest category col dtype
0      air_store_id     314                     0.0                                 2.449717    object
1    visit_datetime    4975                     0.0  

In [95]:
#min and max date time in reservation
min,max=air_reserve.reserve_datetime.min(),air_reserve.reserve_datetime.max()
print(min,max)
print(f"days {(pd.to_datetime(min)-pd.to_datetime(max)).days}")

2016-01-01 01:00:00 2017-04-22 23:00:00
days -478


In [96]:
#min and max date time in visit datetime
min,max=air_reserve.visit_datetime.min(),air_reserve.visit_datetime.max()
print(min,max)
print(f"days {(pd.to_datetime(min)-pd.to_datetime(max)).days}")

2016-01-01 19:00:00 2017-05-31 21:00:00
days -517


In [99]:
#stores present in air stores info and air reserve info
np.setdiff1d(air_reserve.air_store_id,air_store_info.air_store_id)

array([], dtype=object)

<div class="alert alert-info" style="padding:0px 10px; border-radius:5px;"><h3 style='margin:10px 5px'> Inferences:</h3>
</div>

- 314 stores in air data (unique air_store_ids)
- 13325 stores in hpg data
- Missing values are not there in any data
- Both the dataset have 5 columns - id,genre_name,area_name_latitude,longitude
- All the stores in air reserve data have store information in air store info data
- HPG reserve data has more stores compared to hpg store info data. 8635 stores in hpg reserve data doesn't have store information in hpg store info data

#### Undertsnad AIR store and HPG store relation 

In [101]:
understanddata(store_id_relation)

dimesnion of dataset 
(150, 2)


summary of dataset 
                 air_store_id          hpg_store_id
count                    150                   150
unique                   150                   150
top     air_63b13c56b7201bd9  hpg_4bc649e72e2a239a
freq                       1                     1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   air_store_id  150 non-null    object
 1   hpg_store_id  150 non-null    object
dtypes: object(2)
memory usage: 2.5+ KB
None


Statics of dataset are- 

        feature  unique  perc of missing values  perc of values of the  biggest category col dtype
0  air_store_id     150                     0.0                                 0.666667    object
1  hpg_store_id     150                     0.0                                 0.666667    object




In [102]:
understanddata(sample_submission)

dimesnion of dataset 
(32019, 2)


summary of dataset 
        visitors
count   32019.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32019 entries, 0 to 32018
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        32019 non-null  object
 1   visitors  32019 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 500.4+ KB
None


Statics of dataset are- 

    feature  unique  perc of missing values  perc of values of the  biggest category col dtype
0        id   32019                     0.0                                 0.003123    object
1  visitors       1                     0.0                               100.000000     int64




In [103]:
sample_submission.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


In [104]:
sample_submission['visit_date']=sample_submission.apply(lambda x:x['id'].split('_')[2],axis=1)
sample_submission['air_store_id']=sample_submission.apply(lambda x:x['id'].split('_')[0] + "_"+x['id'].split('_')[1],axis=1)
sample_submission['store_type']=sample_submission.apply(lambda x:x['id'].split('_')[0],axis=1)
sample_submission.head()

,id,visitors,visit_date,air_store_id,store_type
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,air
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,air
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,air
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,air
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,air


In [105]:
understanddata(sample_submission)

dimesnion of dataset 
(32019, 5)


summary of dataset 
        visitors
count   32019.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32019 entries, 0 to 32018
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            32019 non-null  object
 1   visitors      32019 non-null  int64 
 2   visit_date    32019 non-null  object
 3   air_store_id  32019 non-null  object
 4   store_type    32019 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.2+ MB
None


Statics of dataset are- 

        feature  unique  perc of missing values  perc of values of the  biggest category col dtype
0            id   32019                     0.0                                 0.003123    object
1      visitors       1                     0.0                               100.000000     int64
2    visit_date 

In [106]:
#datetime need to predict from sample submission data
print(sample_submission.visit_date.min())
print(sample_submission.visit_date.max())

2017-04-23
2017-05-31


In [110]:
#stores present in sample submission data  and not is air store info
print(np.setdiff1d(sample_submission.air_store_id,air_store_info.air_store_id))

#stores present in sample submission data  and not is air visit info
print(np.setdiff1d(sample_submission.air_store_id,air_visit_data.air_store_id))

[]
[]


<div class="alert alert-info" style="padding:0px 10px; border-radius:5px;"><h3 style='margin:10px 5px'> Inferences:</h3>
</div>

- All the predictions needs to be made for air stores,since sample submission has only AIR stores data
- Predictions needs to be made for 821 stores for 39 days each 
- All the stores are present in air store info data and air visit data
- Since predictions needs to be made for air stores, air stores data should not be lost while merging the dataset


<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>4.Data Preprocessing -Phase 1</h2>
</div>rm one

###  The goal of this section is to:

* Merge stores data (air & hpg) to get common information
* Merge reserves data to get common reservation information
* Engineer holiday related features
* Merge all the datasets to create one mega dataset for training
* Let's start with the first task to merge datasets to form one

#### Prepare store info 

In [114]:
air_store_info.shape

(829, 5)

In [113]:
store_id_relation.shape

(150, 2)

In [118]:
store_df=pd.merge(air_store_info,store_id_relation,on='air_store_id',how='left')
store_df.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude,hpg_store_id
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,NaN
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,hpg_9b38b9e13da6da27
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,NaN
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,NaN
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN


How many stores didn't find a matching `hpg_store_id`?

In [122]:
store_df.hpg_store_id.isna().sum()/store_df.shape[0]

0.8190591073582629

In [124]:
hpg_store_info.shape


(4690, 5)

#### add *hpg_store* info 

In [137]:
hpg_store_info

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,hpg_9b38b9e13da6da27,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,hpg_9b38b9e13da6da27,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,hpg_9b38b9e13da6da27,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,hpg_9b38b9e13da6da27,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,hpg_9b38b9e13da6da27,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
...,...,...,...,...,...
4685,hpg_c6ed353a48c7c93e,hpg_9b38b9e13da6da27,Tōkyō-to Chiyoda-ku None,35.695780,139.768453
4686,hpg_a58d2423360b2fb4,hpg_9b38b9e13da6da27,Tōkyō-to Minato-ku Roppongi,35.666137,139.733978
4687,hpg_d2021704076cdd70,hpg_9b38b9e13da6da27,Tōkyō-to Chiyoda-ku None,35.695780,139.768453
4688,hpg_602472570d65e219,hpg_9b38b9e13da6da27,Tōkyō-to Toshima-ku None,35.730068,139.711742


In [127]:
store_df=pd.merge(store_df,hpg_store_info,how='left',on='hpg_store_id',suffixes=['_air','_hpg'])
store_df.shape

(829, 10)

In [140]:
store_df.head()

,air_store_id,air_genre_name,air_area_name,latitude_air,longitude_air,hpg_store_id,hpg_genre_name,hpg_area_name,latitude_hpg,longitude_hpg
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,NaN,NaN,NaN,NaN,NaN
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,hpg_9b38b9e13da6da27,NaN,NaN,NaN,NaN
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,NaN,NaN,NaN,NaN,NaN
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,NaN,NaN,NaN,NaN,NaN
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaN,NaN,NaN


In [141]:
store_df.hpg_genre_name.isnull().sum()

766

#### Prepare reserve data


In [142]:
hpg_reserve.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [143]:
hpg_reserve=pd.merge(hpg_reserve,store_id_relation,on='hpg_store_id')[air_reserve.columns]
print(hpg_reserve.shape)
hpg_reserve.head()

(28183, 4)


,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_db80363d35f10926,2016-01-01 19:00:00,2016-01-01 15:00:00,4
1,air_db80363d35f10926,2016-01-02 19:00:00,2016-01-02 14:00:00,2
2,air_db80363d35f10926,2016-01-03 18:00:00,2016-01-02 20:00:00,6
3,air_db80363d35f10926,2016-01-06 20:00:00,2016-01-04 22:00:00,3
4,air_db80363d35f10926,2016-01-11 18:00:00,2016-01-11 14:00:00,2


In [146]:
air_reserve.shape

(92378, 4)

In [147]:
reserve_df=pd.concat([hpg_reserve,air_reserve],axis=0)
reserve_df.shape

(120561, 4)

#### Prepare Air visits data 

 Create 'id' column in air_visits data, need only those stores ids that we need to forecast

In [150]:
air_visit_data['id']=air_visit_data.air_store_id.str.cat(air_visit_data.visit_date.astype(str),sep='_')

In [153]:
air_visit_data=air_visit_data[["id","air_store_id","visit_date","visitors"]]
air_visit_data

,id,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24_2016-01-13,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24_2016-01-14,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24_2016-01-15,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24_2016-01-16,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24_2016-01-18,air_ba937bf13d40fb24,2016-01-18,6
...,...,...,...,...
252103,air_24e8414b9b07decb_2017-04-18,air_24e8414b9b07decb,2017-04-18,6
252104,air_24e8414b9b07decb_2017-04-19,air_24e8414b9b07decb,2017-04-19,6
252105,air_24e8414b9b07decb_2017-04-20,air_24e8414b9b07decb,2017-04-20,7
252106,air_24e8414b9b07decb_2017-04-21,air_24e8414b9b07decb,2017-04-21,8


#### Submission data

In [155]:
sample_submission=sample_submission[["id",'air_store_id',"visit_date","visitors"]]
sample_submission.head()

,id,air_store_id,visit_date,visitors
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27,0


Airvisits has 252108 rows , but we will kepp only those present in sample submission 

In [156]:
air_visit_data=air_visit_data.loc[air_visit_data.air_store_id.isin(sample_submission.air_store_id),]
air_visit_data.shape

(250468, 4)

concat row-wise , the air visits data and the submission data,keeping only those air idds that are present in submission data, because we care to forecast only those

In [160]:
visits_df=pd.concat([air_visit_data,sample_submission],axis=0,keys=['past','future'],names=['dataset']).reset_index(level='dataset')\
                                                    .reset_index(drop=True)

In [161]:
visits_df

,dataset,id,air_store_id,visit_date,visitors
0,past,air_ba937bf13d40fb24_2016-01-13,air_ba937bf13d40fb24,2016-01-13,25
1,past,air_ba937bf13d40fb24_2016-01-14,air_ba937bf13d40fb24,2016-01-14,32
2,past,air_ba937bf13d40fb24_2016-01-15,air_ba937bf13d40fb24,2016-01-15,29
3,past,air_ba937bf13d40fb24_2016-01-16,air_ba937bf13d40fb24,2016-01-16,22
4,past,air_ba937bf13d40fb24_2016-01-18,air_ba937bf13d40fb24,2016-01-18,6
...,...,...,...,...,...
282482,future,air_fff68b929994bfbd_2017-05-27,air_fff68b929994bfbd,2017-05-27,0
282483,future,air_fff68b929994bfbd_2017-05-28,air_fff68b929994bfbd,2017-05-28,0
282484,future,air_fff68b929994bfbd_2017-05-29,air_fff68b929994bfbd,2017-05-29,0
282485,future,air_fff68b929994bfbd_2017-05-30,air_fff68b929994bfbd,2017-05-30,0


#### Date info

In [162]:
date_info.head()


,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [163]:
#changing the datatype
date_info.rename(columns={"calendar_date":"visit_date"},inplace=True)


In [165]:
date_info['visit_date']=pd.to_datetime(date_info.visit_date,format='%Y-%m-%d')

In [169]:
date_info.head()

,visit_date,day_of_week,holiday_flg,weekend
0,2016-01-01,Friday,1,0
1,2016-01-02,Saturday,1,1
2,2016-01-03,Sunday,1,1
3,2016-01-04,Monday,0,0
4,2016-01-05,Tuesday,0,0


In [168]:
#create a weekedn variable 
date_info['weekend']=[1 if (day=='Saturday' or day=='Sunday') else 0 for day in date_info.day_of_week]